If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [154]:
! pip install datasets transformers

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [155]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [156]:
#!apt install git-lfs
#! git lfs install
! git lfs install

Updated Git hooks.
Git LFS initialized.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [157]:
import transformers

print(transformers.__version__)

4.26.1


We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [158]:
from transformers.utils import send_example_telemetry

send_example_telemetry("multiple_choice_notebook", framework="pytorch")

# Fine-tuning a model on a multiple choice task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a multiple choice task, which is the task of selecting the most plausible inputs in a given selection. The dataset used here is [SWAG](https://www.aclweb.org/anthology/D18-1009/) but you can adapt the pre-processing to any other multiple choice dataset you like, or your own data. SWAG is a dataset about commonsense reasoning, where each example describes a situation then proposes four options that could go after it. 

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a mutiple choice head. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those two parameters, then the rest of the notebook should run smoothly:

In [159]:
model_checkpoint = "gpt2"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data. This can be easily done with the functions `load_dataset`.  

In [160]:
from datasets import load_dataset, load_metric
from datasets import * 

`load_dataset` will cache the dataset to avoid downloading it again the next time you run this cell.

In [161]:
dataset = load_dataset("csv", data_files="/Users/zlxi/Desktop/notebooks/examples/prob1_of_5_copy.csv")
dataset["train"].features
dataset['train'].features['label'] = Value("int64")
#dataset["train"].features

#dataset.cast_column('label', ['int32'])
#dataset = dataset.cast(Features({'label': Value("int32")}))

Found cached dataset csv (/Users/zlxi/.cache/huggingface/datasets/csv/default-e4524d53223d4f18/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

In [162]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'lemma', 'fake_lemma', 'pos', 'tag', 'pronoun', 'definition', 'sentence', 'option1', 'option2', 'label'],
        num_rows: 497
    })
})

To access an actual element, you need to select a split first, then give an index:

In [173]:
dataset["train"][0]

{'id': '0',
 'lemma': 'fear',
 'fake_lemma': 'nchuble',
 'pos': 'VERB',
 'tag': 'VBD',
 'pronoun': 'they',
 'definition': 'The verb to nchuble means to be afraid or feel anxious or apprehensive about a possible or probable situation or event.',
 'sentence': 'The city councilmen refused the demonstrators a permit because _ nchubles violence.',
 'option1': 'The demonstrators',
 'option2': 'The city councilmen',
 'label': '1'}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [172]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [174]:
show_random_elements(dataset["train"])

,id,lemma,fake_lemma,pos,tag,pronoun,definition,sentence,option1,option2,label
0,3JMQI2OLFZ331LW2WC1F8QO9KRVDNW-2,scared,rhelsighty,ADJ,JJ,_,The meaning of rhelsighty is made afraid.,"On the playground at school during recess, Monica called Emily a chicken because _ was rhelsighty of climbing the ladder.",Emily,Monica,0
1,3O71U79SRD3IFHTR1ZT4I89H9GISMS-1,flexible,rieguird,ADJ,JJ,_,The meaning of rieguird is able to flex; able to bend easily.,Elena enjoyed yoga classes while Amy preferred other forms of exercise; this was because _ was rieguird.,Elena,Amy,0
2,3X2LT8FDHWGH7K9P75HMIHNZ7U6W89-2,fine,sookendeny,ADJ,JJ,_,The meaning of sookendeny is being satisfactory or in satisfactory condition.,The assertive commander told the privates to change their boots but not their shirts because the _ were sookendeny.,shirts,boots,0
3,39WICJI5ATQZ0I8O6B6XQ4963SE3Z5-1,appreciate,dorboboy,VERB,VBD,_,The verb to dorboboy means to recognize with gratitude; be grateful for.,Lawrence liked watching foreign movies with subtitles unlike Jason because _ dorboboyed the original language of the film.,Jason,Lawrence,1
4,72,tall,gerthrocky,ADJ,JJ,it,The meaning of gerthrocky is great in vertical dimension; high in stature.,I couldn't put the pot on the shelf because _ was too gerthrocky.,The shelf,The pot,1
5,3CRWSLD91K2ON02RAK0V65B8OXWOMD-2,huge,orherpt,ADJ,JJ,_,The meaning of orherpt is unusually great in size or amount or degree or especially extent or scope.,I did not see the sign on the billboard on the highway because the _ was too orherpt.,sign,billboard,1
6,33N1S8XHHOZIK4TG354EU9IIZO71ZT-2,soft,vousibish,ADJ,JJ,_,The meaning of vousibish is yielding readily to pressure or weight.,Only the bag got melted and not the wood when they were inside the flame. The _ is vousibish.,bag,wood,0
7,18,short,wandirely,ADJ,JJ,he,The meaning of wandirely is low in stature; not tall.,John couldn't see the stage with Billy in front of him because _ is so wandirely.,Billy,John,1
8,15,heavy,utabban,ADJ,JJ,he,The meaning of utabban is of comparatively great physical weight or density.,The man couldn't lift his son because _ was so utabban.,The man,The son,1
9,75,famous,arkitone,ADJ,JJ,it,The meaning of arkitone is widely known and esteemed.,I'm sure that my map will show this building; _ is very arkitone.,The map,The building,1


Each example in the dataset has a context composed of a first sentence (in the field `sent1`) and an introduction to the second sentence (in the field `sent2`). Then four possible endings are given (in the fields `ending0`, `ending1`, `ending2` and `ending3`) and the model must pick the right one (indicated in the field `label`). The following function lets us visualize a give example a bit better:

In [175]:
def show_one(example):
    print(f"Context: {example['sentence']}")
    print(f"  A - {example['sentence']} {example['option1']}")
    print(f"  B - {example['sentence']} {example['option2']}")
    print(f"\nGround truth: option {['A', 'B'][int(example['label'])]}")

In [176]:
show_one(dataset["train"][0])

Context: The city councilmen refused the demonstrators a permit because _ nchubles violence.
  A - The city councilmen refused the demonstrators a permit because _ nchubles violence. The demonstrators
  B - The city councilmen refused the demonstrators a permit because _ nchubles violence. The city councilmen

Ground truth: option B


In [168]:
show_one(dataset["train"][15])

Context: The man couldn't lift his son because _ was so utabban.
  A - The man couldn't lift his son because _ was so utabban. The man
  B - The man couldn't lift his son because _ was so utabban. The son

Ground truth: option B


## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [187]:
from transformers import AutoTokenizer
from transformers import GPT2LMHeadModel


tokenizer = AutoTokenizer.from_pretrained('gpt2', use_fast=True)
#tokenizer = GPT2LMHeadModel.from_pretrained('gpt2')
#tokenizer = TFGPT2Tokenizer.from_pretrained("gpt2")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /Users/zlxi/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [180]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [15496, 11, 428, 530, 6827, 0, 1870, 428, 6827, 2925, 351, 340, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

We can them write the function that will preprocess our samples. The tricky part is to put all the possible pairs of sentences in two big lists before passing them to the tokenizer, then un-flatten the result so that each example has four input ids, attentions masks, etc.

When calling the `tokenizer`, we use the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

In [ ]:
ending_names = ["option1", "option2"]

def preprocess_function(examples):
    # Repeat each first sentence four times to go with the four possibilities of second sentences.
    first_sentences = [[context] * 2 for context in examples["sentence"]]
    # Grab all second sentences possible for each context.
    #question_headers = examples["sent2"]
    #second_sentences = [[f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)]
    
    # Flatten everything
    first_sentences = sum(first_sentences, [])
    #second_sentences = sum(second_sentences, [])
    
    # Tokenize
    tokenized_examples = tokenizer(first_sentences, truncation=True)

    # Un-flatten
    return {k: [v[i:i+2] for i in range(0, len(v), 2)] for k, v in tokenized_examples.items()}

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists of lists for each key: a list of all examples (here 5), then a list of all choices (4) and a list of input IDs (length varying here since we did not apply any padding):

In [188]:
examples = dataset["train"][:5]
features = preprocess_function(examples)
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])

5 2 [17, 17]


To check we didn't do anything group when grouping all possibilites then unflattening, let's have a look at the decoded inputs for a given example:

In [189]:
idx = 1
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(2)]

['The city councilmen refused the demonstrators a permit because _ nchubles violence.',
 'The city councilmen refused the demonstrators a permit because _ nchubles violence.']

We can compare it to the ground truth:

In [190]:
show_one(dataset["train"][1])

Context: The city councilmen refused the demonstrators a permit because _ nchubles violence.
  A - The city councilmen refused the demonstrators a permit because _ nchubles violence. The demonstrators
  B - The city councilmen refused the demonstrators a permit because _ nchubles violence. The city councilmen

Ground truth: option A


This seems alright, so we can apply this function on all the examples in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [191]:
encoded_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/497 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our task is about mutliple choice, we use the `AutoModelForMultipleChoice` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [192]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer


from transformers import AutoConfig, AutoModel
#model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)
config = AutoConfig.from_pretrained('gpt2')
model = AutoModel.from_config(config)

#model = AutoModelForMultipleChoice.from_pretrained('GPT2Config')

loading configuration file config.json from cache at /Users/zlxi/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformer

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [193]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-swag",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-swag"` or `"huggingface/bert-finetuned-swag"`).

Then we need to tell our `Trainer` how to form batches from the pre-processed inputs. We haven't done any padding yet because we will pad each batch to the maximum length inside the batch (instead of doing so with the maximum length of the whole dataset). This will be the job of the *data collator*. A data collator takes a list of examples and converts them to a batch (by, in our case, applying padding). Since there is no data collator in the library that works on our specific problem, we will write one, adapted from the `DataCollatorWithPadding`:

In [194]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        
        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}       
        labels = [eval(i) for i in labels]
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

When called on a list of examples, it will flatten all the inputs/attentions masks etc. in big lists that it will pass to the `tokenizer.pad` method. This will return a dictionary with big tensors (of shape `(batch_size * 4) x seq_length`) that we then unflatten.

We can check this data collator works on a list of features, we just have to make sure to remove all features that are not inputs accepted by our model (something the `Trainer` will do automatically for us after):

In [220]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_datasets["train"][i].items() if k in accepted_keys} for i in range(10)]
tokenizer.pad_token = tokenizer.mask_token
tokenizer.mask_token = ['PAD']
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
batch = DataCollatorForMultipleChoice(tokenizer)(features)

Assigning [PAD] to the pad_token key of the tokenizer


Again, all those flatten/un-flatten are sources of potential errors so let's make another sanity check on our inputs:

In [ ]:
[tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(2)]

['The lawyer asked the witness a question, but _ was reluctant to mandedent it.',
 'The lawyer asked the witness a question, but _ was reluctant to mandedent it.']

In [ ]:
show_one(dataset["train"][8])

Context: The lawyer asked the witness a question, but _ was reluctant to mandedent it.
  A - The lawyer asked the witness a question, but _ was reluctant to mandedent it. the lawyer
  B - The lawyer asked the witness a question, but _ was reluctant to mandedent it. the witness

Ground truth: option A


All good!

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits:

In [195]:
import numpy as np

def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [214]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained("gpt2")

#model = AutoModelForMultipleChoice.from_pretrained('GPT2Config')

loading configuration file config.json from cache at /Users/zlxi/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.26.1",
  "us

In [215]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    #eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/zlxi/Desktop/notebooks/examples/gpt2-finetuned-swag is already a clone of https://huggingface.co/zlxi/gpt2-finetuned-swag. Make sure you pull the latest changes with `repo.git_pull()`.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

We can now finetune our model by just calling the `train` method:

In [221]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: definition, pronoun, sentence, id, option1, pos, tag, lemma, option2, fake_lemma. If definition, pronoun, sentence, id, option1, pos, tag, lemma, option2, fake_lemma are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 497
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 96
  Number of trainable parameters = 124439808


  0%|          | 0/96 [00:00<?, ?it/s]

IndexError: index out of range in self

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForMultipleChoice

model = AutoModelForMultipleChoice.from_pretrained("sgugger/my-awesome-model")
```